# Digital Chemistry: Molecule Representations 🖥️⚛️

*General Chemistry & Cyberinfrastructure Skills Module*

## Learning Objective
Work with and **differentiate** between common computer‐readable chemical representations:
- **SMILES strings**
- **Graph objects** (nodes/edges)
- **Adjacency matrices**
- **XYZ files / Cartesian coordinates**
- …and how to interconvert them in Python.

### Warm‑Up Questions

**WQ‑1.** In one sentence, what *extra* information does an **XYZ file** contain that a **SMILES** string does not? What key information does SMILES keep that XYZ is missing?

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>

**WQ‑2.** The SMILES strings `O=C=O` and `C(=O)=O` both depict CO₂. Why can two strings represent the same molecule? What does this imply about SMILES “uniqueness”?

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>

## Prerequisites
- Python ≥ 3.8
- **RDKit** for cheminformatics
- **networkx** for graph visualisation/manipulation
- **numpy** and **pandas** for matrices / tables

If you’re on Google Colab, run the install cell below first.

In [ ]:
# !pip install rdkit-pypi networkx pandas numpy -q   # ← Uncomment on first run

from rdkit import Chem
from rdkit.Chem import AllChem
import networkx as nx
import numpy as np
import pandas as pd

## Representation Cheat‑Sheet
| Representation | Key idea | Strengths | Limitations |
|----------------|----------|-----------|-------------|
| **SMILES** | Compact line notation describing connectivity & bond order | Human‐typeable, ubiquitous in databases | Loses 3‑D coordinates and sometimes stereochemistry context |
| **Graph** | Nodes = atoms, edges = bonds | Natural for algorithms (shortest path, fingerprints) | Needs extra attributes to capture 3‑D info |
| **Adjacency matrix** | Square matrix; 1 if atoms *i* and *j* bonded | Linear algebra friendly | Grows as *N²*; not human readable |
| **XYZ** | List of atoms + x y z coordinates | Simple 3‑D, universal import/export | No bonding info; separate file stores charge, spin, etc. |


In [ ]:
def mol_to_graph(mol):
    """Convert RDKit Mol → networkx.Graph with atom symbol labels."""
    g = nx.Graph()
    for atom in mol.GetAtoms():
        g.add_node(atom.GetIdx(), symbol=atom.GetSymbol())
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        g.add_edge(i, j, order=str(bond.GetBondType()))
    return g

def mol_to_xyz(mol):
    """Return XYZ string (after 3‑D embedding)."""
    m3d = Chem.AddHs(Chem.Mol(mol))
    AllChem.EmbedMolecule(m3d, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(m3d)
    conf = m3d.GetConformer()
    lines = [str(m3d.GetNumAtoms()), 'Generated by RDKit']
    for atom in m3d.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        lines.append(f"{atom.GetSymbol():<2} {pos.x:>10.4f} {pos.y:>10.4f} {pos.z:>10.4f}")
    return "\n".join(lines)


## Worked Example — Ethanol (*C₂H₅OH*)

In [ ]:
smiles = 'CCO'
mol = Chem.MolFromSmiles(smiles)
print('SMILES:', smiles)

In [ ]:
adj = Chem.GetAdjacencyMatrix(mol)
df_adj = pd.DataFrame(adj, columns=[a.GetSymbol()+str(a.GetIdx()) for a in mol.GetAtoms()],
                      index=[a.GetSymbol()+str(a.GetIdx()) for a in mol.GetAtoms()])
df_adj

In [ ]:
G = mol_to_graph(mol)
print(f'Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges')
print('Edges with bond order:')
for u, v, d in G.edges(data=True):
    print(f"{G.nodes[u]['symbol']}{u} – {G.nodes[v]['symbol']}{v}: order {d['order']}")

In [ ]:
xyz_text = mol_to_xyz(mol)
print(xyz_text)

## Your Turn 📝
1. Supply **two** SMILES strings.  
2. For each:
   - Show the SMILES.
   - Build and display the adjacency matrix.
   - Convert to a NetworkX graph and print basic stats (nodes, edges).
   - Generate XYZ coordinates.
3. Compare: what information is preserved or lost when moving between representations?

### Auto‑Graded Checkpoints

**Checkpoint CP‑1 (2 pts)** — Implement `adjacency_size(smiles)` in the next cell. It must return `(num_atoms, num_bonds)` for the molecule.

In [ ]:
### BEGIN SOLUTION
from rdkit import Chem

def adjacency_size(smiles):
    mol = Chem.MolFromSmiles(smiles)
    n_atoms = mol.GetNumAtoms()
    n_bonds = mol.GetNumBonds()
    return n_atoms, n_bonds
### END SOLUTION

In [ ]:
# hidden tests
from rdkit import Chem
assert adjacency_size('C') == (5,4)
assert adjacency_size('c1ccccc1') == (12,12)

**Checkpoint CP‑2 (3 pts)** — Implement `tanimoto_self(smiles_list)` to return a square numpy similarity matrix using Morgan fingerprints. The diagonal must be 1.0.

In [ ]:
### BEGIN SOLUTION
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors, DataStructs

def tanimoto_self(smiles_list):
    fps = [rdMolDescriptors.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(s),2,1024) for s in smiles_list]
    n = len(fps)
    mat = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = DataStructs.TanimotoSimilarity(fps[i], fps[j])
    return mat
### END SOLUTION

In [ ]:
# hidden tests
import numpy as np
sm = ['CCO','O=C=O']
M = tanimoto_self(sm)
assert M.shape == (2,2)
assert np.allclose(np.diag(M),1.0)
assert 0 <= M[0,1] <= 1.0

In [ ]:
# TODO 1: Replace with your SMILES
my_smiles = ['O=C=O', 'c1ccccc1']  # CO2, benzene

for smi in my_smiles:
    mol = Chem.MolFromSmiles(smi)
    print('\nSMILES:', smi)
    # TODO 2: Adjacency matrix
    # TODO 3: Graph stats
    # TODO 4: XYZ coords
    pass  # ← your code here


### Challenge ⭐ — SMILES ↔ Graph ↔ SMILES Round‑Trip
Write a function that:
1. Converts a SMILES string to a NetworkX graph, **then**
2. Reconstructs a new RDKit Mol from that graph, and
3. Retrieves a SMILES again.

Compare the original and reconstructed SMILES—are they identical? *(Hint: RDKit’s `RWMol` class and `Chem.MolFromSmiles` validation tools may help.)*

In [ ]:
# TODO 5: Implement round‑trip converter (optional)
def smiles_round_trip(smi):
    pass

## Molecular Descriptors & Fingerprints 🧮🔑

Chemical *descriptors* are numeric summaries of molecular properties, while *fingerprints* are bit‑vectors that encode structural motifs for rapid similarity searches.

| Category | Examples | Typical use |
|----------|----------|-------------|
| **Descriptors (scalar)** | Molecular weight, logP, topological polar surface area (TPSA), number of H‑bond donors/acceptors | QSAR, drug‑likeness filters |
| **Fingerprints (bit‑vector)** | Morgan (ECFP), MACCS, RDKit topological | Substructure search, clustering, similarity metrics |


In [ ]:
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors as rdmd

def basic_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return {
        'MolWt': Descriptors.MolWt(mol),
        'logP': Descriptors.MolLogP(mol),
        'TPSA': rdmd.CalcTPSA(mol),
        'HBA': rdmd.CalcNumLipinskiHBA(mol),
        'HBD': rdmd.CalcNumLipinskiHBD(mol)
    }

# Demo on ethanol
print(basic_descriptors('CCO'))

In [ ]:
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs

ethanol = Chem.MolFromSmiles('CCO')
benzene = Chem.MolFromSmiles('c1ccccc1')

# Morgan fingerprint (radius 2, 1024 bits)
fp_eth = rdmd.GetMorganFingerprintAsBitVect(ethanol, radius=2, nBits=1024)
fp_ben = rdmd.GetMorganFingerprintAsBitVect(benzene, radius=2, nBits=1024)
sim = DataStructs.TanimotoSimilarity(fp_eth, fp_ben)
print(f'Tanimoto similarity (ethanol vs benzene): {sim:.2f}')

# MACCS keys example
maccs_eth = MACCSkeys.GenMACCSKeys(ethanol)
print('MACCS bits set for ethanol:', list(maccs_eth.GetOnBits())[:10], '...')

### Your Turn 📝 — Descriptor & Fingerprint Playground
1. Pick **three** molecules (SMILES).  
2. Build a **pandas DataFrame** of their basic descriptors using `basic_descriptors`.  
3. Compute Morgan fingerprints and create a **similarity matrix** (Tanimoto) between all pairs.  
4. Which pair is most similar? Does that align with chemical intuition?

In [ ]:
# TODO 6: Descriptor & fingerprint analysis
my_smiles = ['CCO', 'O=C=O', 'CC(=O)O']  # edit!

## Build descriptor table
desc_rows = []
for smi in my_smiles:
    row = basic_descriptors(smi)
    row['SMILES'] = smi
    desc_rows.append(row)
df_desc = pd.DataFrame(desc_rows).set_index('SMILES')
display(df_desc)

## Fingerprint similarity matrix
fps = [rdmd.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(s), 2, 1024) for s in my_smiles]
n = len(fps)
sim_mat = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        sim_mat[i, j] = DataStructs.TanimotoSimilarity(fps[i], fps[j])
df_sim = pd.DataFrame(sim_mat, index=my_smiles, columns=my_smiles)
display(df_sim)

### Critical‑Thinking Questions

**CTQ‑1.** For each task below, pick the *best* representation (SMILES, NetworkX graph, adjacency matrix, or XYZ) **and justify**:
a) Counting rings b) Running DFT geometry optimisation c) Fingerprint‑based similarity search

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>

**CTQ‑2.** RDKit’s ETKDG algorithm guesses 3‑D coordinates from SMILES. List two possible *errors* this guess can introduce, and explain how they might mislead downstream ML models.

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>

## Summary & Next Steps
- **SMILES, graphs, adjacency matrices, XYZ**: complementary ways to encode structure.  
- **Descriptors** give *numeric* property snapshots; **fingerprints** give *bitwise* structural signatures for fast similarity.  
- Combining these representations is foundational for QSAR modeling, virtual screening, and cheminformatics pipelines.